# FastAPI Middleware Chain Processing

This notebook explains how FastAPI handles middleware chains and request/response flows.

In [ ]:
# NOTE: This notebook explains FastAPI's middleware chain processing,
#   so even if that logic isn't currently mentioned in the codebase,
#   it could prove useful in the future if we need to add multiple middlewares.

Sources:
1. Trial and error :]
2. Github Copilot :]]
3. This great blog post: Setting Up Middleware in FastAPI
   1. https://www.oneloop.ai/blog/setting-up-middleware-in-fastapi
4. This amazing GitHub comment that traces middleware chain processing:
   1. https://github.com/encode/starlette/issues/479#issuecomment-1595113897
   2. Hint: I suggest seeing this once you understand the `inheriting_middleware_class.ipynb` file.
5. FastAPI docs:
   1. https://fastapi.tiangolo.com/tutorial/middleware/

# Understanding FastAPI Middleware Execution Order

Below is a minimal example that demonstrates the order of middleware execution:

In [ ]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import logging

app = FastAPI()

# Register middlewares in this order
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.middleware("http")
async def second_middleware(request: Request, call_next):
    logging.debug("Before call_next, Currently in func. second_middleware")
    response = await call_next(request)
    logging.debug("After call_next, Currently in func. second_middleware")
    return response


@app.middleware("http")
async def third_middleware(request: Request, call_next):
    logging.debug("Before call_next, Currently in func. third_middleware")
    response = await call_next(request)
    logging.debug("After call_next, Currently in func. third_middleware")
    return response

When a request is received, the log output will be:

```
2024-01-09 10:15:23 | DEBUG | Before call_next, Currently in func. third_middleware
2024-01-09 10:15:23 | DEBUG | Before call_next, Currently in func. second_middleware
2024-01-09 10:15:23 | DEBUG | After call_next, Currently in func. second_middleware
2024-01-09 10:15:23 | DEBUG | After call_next, Currently in func. third_middleware
```

This demonstrates that:
1. Middleware executes in reverse registration order for requests
2. But executes in registration order for responses


So to summarize, the execution flow is as follows:

1. In request: Middleware executes in REVERSE registration order
2. (A hidden step occurs explained between "(pause)" and "(continue)" visualized in the diagram below)
3. In response: Middleware executes in registration order

Visualization of the execution flow:


```
Client Request
     │
     ▼
┌─────────────────┐
│third_middleware │
└─────────────────┘
     │  ┌──────────────────────────────┐
     │  │ ↙ await call_next(request)   │
     │  └──────────────────────────────┘
     ▼
┌──────────────────┐
│second_middleware │
└──────────────────┘
     │  ┌──────────────────────────────┐
     │  │ ↙ await call_next(request)   │
     │  └──────────────────────────────┘
     ▼
┌────────────────┐
│  CORSMiddleware│
└────────────────┘
     │              ...
     │              ...
     │              ...
     │              ...
————————————————————————————————————————————————————————————————————————————— (pause...)

HOWEVER, before passing request (i.e., scope/receive) to `Endpoint` , 
the request is first checked by the middlewares in the ACTUAL registration order 
(I.e., `receive`/`scope` methods are ran (see "source 4" above)) 

TL;DR: This happens:

┌────────────────┐
│  CORSMiddleware│  (checks request) [footnote 1.]
└────────────────┘
     │
     ▼
┌──────────────────┐
│second_middleware │ (checks request)
└──────────────────┘
     │
     ▼
┌─────────────────┐
│third_middleware │  (checks request) 
└─────────────────┘

Then, if all good, then you can imagine the flow before the (pause) gets connected with that after the (continue).
In other words, the `await call_next` in `CORSMiddleware` will call the `Endpoint`,
then the endpoint will process the request and return a response, 
then the Middleware calls will be done in the ACTUAL registration order.
(Side note: If you don't understand, try reading this paragraph after understanding "source 4" above).

TL;DR: we continue from where we paused ▼

————————————————————————————————————————————————————————————————————————————— (continue...)
     │              ...
     │              ...
     │  ┌──────────────────────────────┐
     │  │ ↙ await call_next(request)   │
     │  └──────────────────────────────┘
     │
     ▼
┌────────────────┐
│    Endpoint    │ (processes request, returns response)
└────────────────┘
     │
     ▼
┌────────────────┐
│  CORSMiddleware│
└────────────────┘
     │
     │
     ▼           
┌──────────────────┐  
│second_middleware │ (DEBUG: "After call_next, Currently in func. second_middleware")
└──────────────────┘
     │
     ▼           
┌─────────────────┐ 
│third_middleware │ (DEBUG: "After call_next, Currently in func. third_middleware")
└─────────────────┘
     │
     ▼
Client Response
```

This visualization shows:
1. Request flows down through middlewares in reverse registration order
2. Each middleware logs before passing the request down
3. Response flows back up through middlewares in registration order
4. Each middleware logs after receiving the response

The `call_next` function is what enables this chain-like behavior, passing control to the next middleware in line.

Side note:

Regarding the "[footnote 1.]" above: 
* If an error occurs at that place, then it will be caught by its preceding `second_middleware()` (assuming it has a try/except block)
  